This notebook contains the documentation of the errors, mishaps, tips, tricks, and solutions found in the process of developing a comprehensive python control scripts for the various hardware components of the Pulsed Laser Depostion System.

## Laser Attenuator


## XPS Motion Stage Controller


## Alicat Mass Flow Controllers(2)


## Neocera Target Carousel Controller


Testing Python control script built entirely around ticcmd.

In [15]:
class TicCmdController:
    """
    High-level interface for controlling a single Tic stepper motor controller.
    Wraps common operations like listing devices, reading status, and setting position.
    """

    def __init__(self, serial=None):
        """
        Initializes the controller for a specific Tic device.

        Args:
            serial (str, optional): Serial number of the Tic device. If None, attempts to auto-select the first found device.
        
        Raises:
            RuntimeError: If no connected Tic devices are found.
        """
        self.serial = serial or self._get_first_serial()
        if self.serial is None:
            raise RuntimeError("No Tic device found.")

    @staticmethod
    def list_devices():
        """
        Lists all connected Tic devices by serial number.

        Returns:
            list of str: Serial numbers of detected devices.
        """
        try:
            output = TicCmd.run("--list")
            devices = []
            for line in output.strip().split("\n"):
                if line.strip():
                    serial = line.split(",")[0].strip()
                    devices.append(serial)
            return devices
        except Exception as e:
            print("Failed to list Tic devices:", e)
            return []

    def _get_first_serial(self):
        """
        Returns the serial number of the first detected Tic device.

        Returns:
            str: Serial number of the first device, or None if none are found.
        """
        devices = self.list_devices()
        return devices[0] if devices else None

    def get_status(self, full=True, parsed=True):
        """
        Retrieves the current status of the motor.

        Args:
            full (bool): Whether to use `--full` for more detailed YAML output (default: True).
            parsed (bool): If True, returns parsed YAML dictionary. If False, returns raw output string.

        Returns:
            dict or str: Device status information.
        """
        args = ["--status"]
        if full:
            args.append("--full")

        output = TicCmd.run_with_serial(self.serial, *args)
        if parsed:
            return yaml.safe_load(output)
        return output

    def set_target_position(self, position):
        """
        Sends a command to move the motor to a specific position.

        Args:
            position (int): Target position in microsteps.

        Raises:
            Exception: If the command fails.
        """
        try:
            TicCmd.run_with_serial(
                self.serial,
                "--exit-safe-start",
                "--position", str(position),
                capture_output=False
            )
        except Exception as e:
            print(f"Failed to move motor {self.serial} to position {position}")


In [ ]:
class TicControllerManager:
    def __init__(self, serial_map):
        """
        Args:
            serial_map (dict): Maps logical names to device serials.
                Example: {"raster_x": "00475586", "raster_y": "00475522"}
        """
        self.controllers = {
            name: TicCmdController(serial)
            for name, serial in serial_map.items()
        }

    def get(self, name):
        """
        Retrieve a TicCmdController by logical name.
        """
        return self.controllers[name]

    def all(self):
        """
        Iterator over (name, controller) pairs.
        """
        return self.controllers.items()

    def get_all_serials(self):
        return [ctrl.serial for ctrl in self.controllers.values()]


Usage Tests

In [12]:
motor1 = TicCmdController()
motor1.list_devices()
motor1.get_status(full=True)

Name:                         Tic T249 Stepper Motor Controller
Serial number:                00475586
Firmware version:             1.06
Last reset:                   Power-on reset
Up time:                      51:59:31

Encoder position:             0
RC pulse width:               N/A
Input state:                  Not ready
Input after averaging:        N/A
Input after hysteresis:       N/A
Input before scaling:         N/A
Input after scaling:          0
Forward limit active:         No
Reverse limit active:         No

VIN voltage:                  23.953 V
Operation state:              De-energized
Energized:                    No
Homing active:                No
Last motor driver error:      None

Target:                       No target
Current position:             0
Position uncertain:           Yes
Current velocity:             0
Max speed:                    2000000
Starting speed:               0
Max acceleration:             40000
Max deceleration:             40000
Acting

In [10]:
serials = {
        "raster_x": "00475586",
        "raster_y": "00475522"
    }

In [11]:
manager = TicControllerManager(serials)

# Single device access
manager.get("raster_x").get_status()

# All devices
for name, ctrl in manager.all():
    print(f"--- Status for {name} ({ctrl.serial}) ---")
    ctrl.get_status()

Name:                         Tic T249 Stepper Motor Controller
Serial number:                00475586
Firmware version:             1.06
Last reset:                   Power-on reset
Up time:                      28:10:22

Encoder position:             0
Input state:                  Not ready
Input after averaging:        N/A
Input after hysteresis:       N/A
Input before scaling:         N/A
Input after scaling:          0
Forward limit active:         No
Reverse limit active:         No

VIN voltage:                  24.0 V
Operation state:              De-energized
Energized:                    No
Homing active:                No
Last motor driver error:      None

Target:                       No target
Current position:             0
Position uncertain:           Yes
Current velocity:             0

Errors currently stopping the motor:
  - Intentionally de-energized
  - Serial error
  - Command timeout
  - Safe start violation
Errors that occurred since last check:
  - Command ti

## Pfeiffer Vacuum Manometer


## MKS 651C Throttle Valve Pressure Controller 


## LAMBDA Physik LPX200 Laser Controls